In [1]:
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from helpers import *
import yfinance as yf

/Users/irfanjamil/anaconda3/lib/python3.10/site-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [3]:
import time

In [4]:
cursor = get_cursor()

In [5]:
from KalshiClientsBaseV2 import ExchangeClient

In [6]:
ndx = yf.Ticker('^NDX').history(period='max')
spx = yf.Ticker('^SPX').history(period='max')

spx = spx.reset_index()[['Date', 'Open', 'Close']]
ndx = ndx.reset_index()[['Date', 'Open', 'Close']]

spx['Date'] = spx.apply(lambda x: date(x['Date'].year, x['Date'].month, x['Date'].day), axis=1)
ndx['Date'] = ndx.apply(lambda x: date(x['Date'].year, x['Date'].month, x['Date'].day), axis=1)

In [7]:
spx[spx['Date']==date(2022,5,2)]

,Date,Open,Close
23696,2022-05-02,4130.609863,4155.379883


In [8]:
prod_email, prod_password = get_kalshi_creds()
prod_api_base = "https://trading-api.kalshi.com/trade-api/v2" 
exchange_client = ExchangeClient(exchange_api_base=prod_api_base, email = prod_email, password = prod_password)

In [9]:
abbr_to_month = {v: k for k, v in month_to_abbr.items()}

In [10]:
exchange_client.get_market_history('INX-22MAY02-B4125')

{'ticker': 'INX-22MAY02-B4125',
 'history': [{'yes_price': 0,
   'yes_bid': 20,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 80,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651427004},
  {'yes_price': 0,
   'yes_bid': 21,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 79,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651427004},
  {'yes_price': 0,
   'yes_bid': 22,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 78,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651427004},
  {'yes_price': 0,
   'yes_bid': 23,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 77,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651427004},
  {'yes_price': 0,
   'yes_bid': 24,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 76,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651427004},
  {'yes_price': 0,
   'yes_bid': 24,
   'yes_ask': 28,
   'no_bid': 72,
   'no_ask': 76,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651427004},
  {'yes_price': 0,
   'yes_bid': 24,
   'yes_ask': 28,
  

In [11]:
from KalshiClientsBaseV2 import HttpError

In [12]:
#Kalshi SP500 range market goes back to April 28, 2022, size of each range change from 50 to 25 after May 17, 2023
import math
dt = date(2022,4,28)
lb = None
ub = None
mp = None
spx_d = {'ticker': [], 'bid': [], 'ask': [], 'lb': [], 'ub': [], 'open': [], 'close': [], 'date': []}
while dt < date.today():
    retry = False
    try:
        spx_open = float(spx[spx['Date']==dt]['Open'].to_list()[0])
        spx_close = float(spx[spx['Date']==dt]['Close'].to_list()[0])
    except IndexError:
        dt += timedelta(days = 1)
        continue
    if dt <= date(2023, 5, 17):
        lb = spx_open - spx_open % 50
        ub = lb + 50
        mp = int((lb+ub)/2)
    else:
        lb = spx_open - spx_open % 25
        ub = lb + 25
        mp = math.floor((lb + ub)/2)
    ticker = 'INX-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        ticker += '0' + str(dt.day)
    else:
        ticker += str(dt.day)
    ticker += '-B' + str(mp)
    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 14, 30, 0).timetuple()))
    try:
        r = exchange_client.get_market_history(ticker, min_ts = ts)
    except HttpError:
        ticker = ticker[:3] + 'D' + ticker[3:]
        retry = True

    if retry:
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except:
            dt += timedelta(days = 1)
            continue
    info = r['history'][0]
    first_ts = datetime.fromtimestamp(info['ts'])
    if first_ts.hour == 14:
        bb = info['yes_bid']
        ba = info['yes_ask']
        spx_d['ticker'].append(ticker)
        spx_d['bid'].append(bb)
        spx_d['ask'].append(ba)
        spx_d['lb'].append(lb)
        spx_d['ub'].append(ub)
        spx_d['date'].append(dt)        
        spx_d['open'].append(spx_open)
        spx_d['close'].append(spx_close)

    dt += timedelta(days=1)
spx_df = pd.DataFrame(data=spx_d)

In [13]:
spx_df

,ticker,bid,ask,lb,ub,open,close,date
0,INX-22APR28-B4225,15,19,4200.0,4250.0,4222.580078,4287.500000,2022-04-28
1,INX-22APR29-B4275,0,4,4250.0,4300.0,4253.750000,4131.930176,2022-04-29
2,INX-22MAY02-B4125,21,25,4100.0,4150.0,4130.609863,4155.379883,2022-05-02
3,INX-22MAY03-B4175,40,44,4150.0,4200.0,4159.779785,4175.479980,2022-05-03
4,INX-22MAY04-B4175,42,45,4150.0,4200.0,4181.180176,4300.169922,2022-05-04
...,...,...,...,...,...,...,...,...
367,INX-24FEB13-B4962,14,15,4950.0,4975.0,4967.939941,4953.169922,2024-02-13
368,INX-24FEB14-B4987,56,58,4975.0,5000.0,4976.439941,5000.620117,2024-02-14
369,INX-24FEB15-B5012,44,45,5000.0,5025.0,5003.140137,5029.729980,2024-02-15
370,INX-24FEB16-B5037,69,71,5025.0,5050.0,5031.129883,5005.569824,2024-02-16


In [14]:
spx_df['in_interval'] = spx_df.apply(lambda x: 1 if x['close']>= x['lb'] and x['close']<x['ub'] else 0, axis=1)
spx_df

,ticker,bid,ask,lb,ub,open,close,date,in_interval
0,INX-22APR28-B4225,15,19,4200.0,4250.0,4222.580078,4287.500000,2022-04-28,0
1,INX-22APR29-B4275,0,4,4250.0,4300.0,4253.750000,4131.930176,2022-04-29,0
2,INX-22MAY02-B4125,21,25,4100.0,4150.0,4130.609863,4155.379883,2022-05-02,0
3,INX-22MAY03-B4175,40,44,4150.0,4200.0,4159.779785,4175.479980,2022-05-03,1
4,INX-22MAY04-B4175,42,45,4150.0,4200.0,4181.180176,4300.169922,2022-05-04,0
...,...,...,...,...,...,...,...,...,...
367,INX-24FEB13-B4962,14,15,4950.0,4975.0,4967.939941,4953.169922,2024-02-13,1
368,INX-24FEB14-B4987,56,58,4975.0,5000.0,4976.439941,5000.620117,2024-02-14,0
369,INX-24FEB15-B5012,44,45,5000.0,5025.0,5003.140137,5029.729980,2024-02-15,0
370,INX-24FEB16-B5037,69,71,5025.0,5050.0,5031.129883,5005.569824,2024-02-16,0


In [15]:
spx_df['in_interval'].describe()

count    372.00000
mean       0.41129
std        0.49273
min        0.00000
25%        0.00000
50%        0.00000
75%        1.00000
max        1.00000
Name: in_interval, dtype: float64

In [ ]:
spx_df[spx_df['bid']>50]['in_int']

In [21]:
exchange_client.get_market_history('NASDAQ100-22MAY02-B13050')

{'ticker': 'NASDAQ100-22MAY02-B13050',
 'history': [{'yes_price': 0,
   'yes_bid': 5,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 95,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651423399},
  {'yes_price': 0,
   'yes_bid': 6,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 94,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651423399},
  {'yes_price': 0,
   'yes_bid': 7,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 93,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651423399},
  {'yes_price': 0,
   'yes_bid': 8,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 92,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651423399},
  {'yes_price': 0,
   'yes_bid': 9,
   'yes_ask': 100,
   'no_bid': 0,
   'no_ask': 91,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651423399},
  {'yes_price': 0,
   'yes_bid': 9,
   'yes_ask': 12,
   'no_bid': 88,
   'no_ask': 91,
   'volume': 0,
   'open_interest': 0,
   'ts': 1651423399},
  {'yes_price': 0,
   'yes_bid': 9,
   'yes_ask': 12,
  

In [24]:
#Kalshi NASDAQ100 range market goes back to May 1, 2022, size of each range change is 100
import math
dt = date(2022,5,1)
lb = None
ub = None
mp = None
ndx_d = {'ticker': [], 'bid': [], 'ask': [], 'lb': [], 'ub': [], 'open': [], 'close': [], 'date': []}
while dt < date.today():
    retry = False
    try:
        ndx_open = float(ndx[ndx['Date']==dt]['Open'].to_list()[0])
        ndx_close = float(ndx[ndx['Date']==dt]['Close'].to_list()[0])
    except IndexError:
        dt += timedelta(days = 1)
        continue
   
    lb = ndx_open - ndx_open % 100
    ub = lb + 100
    mp = int((lb + ub)/2)
    ticker = 'NASDAQ100-' + str(dt.year-2000) + month_to_abbr[dt.month]
    if dt.day < 10:
        ticker += '0' + str(dt.day)
    else:
        ticker += str(dt.day)
    ticker += '-B' + str(mp)
    ts = int(time.mktime(datetime(dt.year, dt.month, dt.day, 14, 30, 0).timetuple()))
    try:
        r = exchange_client.get_market_history(ticker, min_ts = ts)
    except HttpError:
        ticker = ticker[:9] + 'D' + ticker[9:]
        retry = True

    if retry:
        try:
            r = exchange_client.get_market_history(ticker, min_ts = ts)
        except:
            dt += timedelta(days = 1)
            continue
    info = r['history'][0]
    first_ts = datetime.fromtimestamp(info['ts'])
    if first_ts.hour == 14:
        bb = info['yes_bid']
        ba = info['yes_ask']
        ndx_d['ticker'].append(ticker)
        ndx_d['bid'].append(bb)
        ndx_d['ask'].append(ba)
        ndx_d['lb'].append(lb)
        ndx_d['ub'].append(ub)
        ndx_d['date'].append(dt)        
        ndx_d['open'].append(ndx_open)
        ndx_d['close'].append(ndx_close)

    dt += timedelta(days=1)
ndx_df = pd.DataFrame(data=ndx_d)

In [25]:
ndx_df

,ticker,bid,ask,lb,ub,open,close,date
0,NASDAQ100-22MAY02-B12850,21,25,12800.0,12900.0,12864.830078,13075.849609,2022-05-02
1,NASDAQ100-22MAY03-B13050,31,35,13000.0,13100.0,13045.349609,13089.900391,2022-05-03
2,NASDAQ100-22MAY04-B13150,26,30,13100.0,13200.0,13110.299805,13535.709961,2022-05-04
3,NASDAQ100-22MAY05-B13350,1,3,13300.0,13400.0,13344.790039,12850.549805,2022-05-05
4,NASDAQ100D-22MAY09-B12450,7,9,12400.0,12500.0,12465.910156,12187.719727,2022-05-09
...,...,...,...,...,...,...,...,...
361,NASDAQ100-24FEB13-B17550,57,58,17500.0,17600.0,17519.800781,17600.419922,2024-02-13
362,NASDAQ100-24FEB14-B17750,50,51,17700.0,17800.0,17728.900391,17807.630859,2024-02-14
363,NASDAQ100-24FEB15-B17850,65,66,17800.0,17900.0,17815.320312,17845.720703,2024-02-15
364,NASDAQ100-24FEB16-B17850,57,62,17800.0,17900.0,17853.589844,17685.980469,2024-02-16


In [27]:
ndx_df['in_interval'] = ndx_df.apply(lambda x: 1 if x['close'] >= x['lb'] and x['close'] < x['ub'] else 0, axis=1)

In [28]:
ndx_df

,ticker,bid,ask,lb,ub,open,close,date,in_interval
0,NASDAQ100-22MAY02-B12850,21,25,12800.0,12900.0,12864.830078,13075.849609,2022-05-02,0
1,NASDAQ100-22MAY03-B13050,31,35,13000.0,13100.0,13045.349609,13089.900391,2022-05-03,1
2,NASDAQ100-22MAY04-B13150,26,30,13100.0,13200.0,13110.299805,13535.709961,2022-05-04,0
3,NASDAQ100-22MAY05-B13350,1,3,13300.0,13400.0,13344.790039,12850.549805,2022-05-05,0
4,NASDAQ100D-22MAY09-B12450,7,9,12400.0,12500.0,12465.910156,12187.719727,2022-05-09,0
...,...,...,...,...,...,...,...,...,...
361,NASDAQ100-24FEB13-B17550,57,58,17500.0,17600.0,17519.800781,17600.419922,2024-02-13,0
362,NASDAQ100-24FEB14-B17750,50,51,17700.0,17800.0,17728.900391,17807.630859,2024-02-14,0
363,NASDAQ100-24FEB15-B17850,65,66,17800.0,17900.0,17815.320312,17845.720703,2024-02-15,1
364,NASDAQ100-24FEB16-B17850,57,62,17800.0,17900.0,17853.589844,17685.980469,2024-02-16,0


In [29]:
ndx_df['in_interval'].describe()

count    366.000000
mean       0.265027
std        0.441952
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: in_interval, dtype: float64

In [ ]:
def get_interval_bounds(s):
    split = s.split(' ')
    if 'or' in s and 'below' in s:
      return [0, float(split[0].replace(',',''))]
    elif 'or' in s and 'above' in s:
      return [float(split[0].replace(',','')), np.inf]
    elif 'to' in s:
      return [float(split[0].replace(',','')), float(split[2].replace(',',''))]
    else:
      raise Exception
    

In [ ]:
cursor.execute('select Date, Open, Close from spx;')
spx_rows = cursor.fetchall()
spx_d = {'Date': [], 'Open': [], 'Close': []}

cursor.execute('select Date, Open, Close from ndx;')
ndx_rows = cursor.fetchall()
ndx_d = {'Date': [], 'Open': [], 'Close': []}

for row in spx_rows:
    spx_d['Date'].append(row[0])
    spx_d['Open'].append(row[1])
    spx_d['Close'].append(row[2])
    
for row in ndx_rows:
    ndx_d['Date'].append(row[0])
    ndx_d['Open'].append(row[1])
    ndx_d['Close'].append(row[2])

spx = pd.DataFrame(data=spx_d)
ndx = pd.DataFrame(data=ndx_d)

In [ ]:

d = {'ticker': [], 'bid': [], 'ask': [], 'lb': [], 'ub': [], 'open': [], 'close': [], 'date': []}
for ticker in tickers:
    ticker_dt = ticker.split('-')[1]
    year = 2000 + int(ticker_dt[:2])
    day = int(ticker_dt[-2:])
    month = abbr_to_month[ticker_dt[2:5]]
    ts = int(time.mktime(datetime(year, month, day, 14, 30, 0).timetuple()))
    r = exchange_client.get_market_history(ticker, min_ts=ts)
    info = r['history'][0]
    first_ts = datetime.fromtimestamp(info['ts'])
    if first_ts.hour == 14:
        bb = info['yes_bid']
        ba = info['yes_ask']
        get_mark_resp = exchange_client.get_market(ticker)['market']
        subtitle = get_mark_resp['subtitle']
        ticker_date = datetime.strptime(get_mark_resp['close_time'], '%Y-%m-%dT%H:%M:%SZ').date()
        d['ticker'].append(ticker)
        d['bid'].append(bb)
        d['ask'].append(ba)
        interval = get_interval_bounds(subtitle)
        d['lb'].append(interval[0])
        d['ub'].append(interval[1])
        d['date'].append(ticker_date)
        if 'INX' in ticker:
            row = spx[spx['Date']==ticker_date]
            if row.shape[0]>0:
                d['open'].append(float(row['Open'].to_list()[0]))
                d['close'].append(float(row['Close'].to_list()[0]))
            else:
                d['open'].append(None)
                d['close'].append(None)
        else:
            row = ndx[ndx['Date']==ticker_date]
            if row.shape[0]>0:
                d['open'].append(float(row['Open'].to_list()[0]))
                d['close'].append(float(row['Close'].to_list()[0]))
            else:
                d['open'].append(None)
                d['close'].append(None)
        

df = pd.DataFrame(data=d)

In [ ]:
df[(df['open'].isnull()==False) & (df['open'] >= df['lb']) & (df['open']<=df['ub'])]

In [ ]:
spx